In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import choice,normal
from scipy.optimize import curve_fit

from ipywidgets import IntProgress
from IPython.display import display

# Simulation setups

In [ ]:
# --- e-f Rabi oscillation physics simulation setup ---
f01 = 5.0e9
anharmonicity = -0.25e9

TRabi = 250e-9 #Rabi oscillation period
wRabi = 2*np.pi/TRabi 

def rabi_sim(_p0,_t): #Takes the initial state probabilities [p0g,p0e,p0f=0] and the pulse duration/driving time of the Rabi oscillation and returns the probabilities of the states in that moment.
    global wRabi
    return [_p0[0],(_p0[1]/2)*(1+np.cos(wRabi*_t)),(_p0[1]/2)*(1-np.cos(wRabi*_t))]

In [ ]:
# --- Transmon readout stochastic simulation setup ---
# SingleTransmonTestBackend calibrations for the IQ state normal distributions
gI_mean = 9.9 -10
gI_sd = 1.9
gQ_mean = 0.0 -10
gQ_sd = 2.1

eI_mean = -5.1 -10
eI_sd = 2.0
eQ_mean = 8.7 -10
eQ_sd = 2.0

fI_mean = -4.9 -10
fI_sd = 2.0
fQ_mean = -8.7 -10
fQ_sd = 2.0
# The readout_sim function taker the states probabilities pg,pe,pf, the number of shots and the ouput_type ('single' or 'mean') returns in each case: 
# 1) 'single': a tuple, first a list as long as the number of shots requested in which element is a 2-element array [I,Q], second the standard deviations determined by the distribution.
# 2) 'mean': a tuple, first 2-element array [I,Q] which is the mean of the number of shots itaration, i.e., the mean of the 'single' output-type; second the standard deviations calculated by taken the sd of the mean of samples.
def readout_sim(_p,_shots,_output_type):
    global gI_mean, gI_sd, gQ_mean, gQ_sd, eI_mean, eI_sd, eQ_mean, eQ_sd, fI_mean, fI_sd, fQ_mean, fQ_sd
    _sample = choice([0,1,2], _shots, p=[_p[0],_p[1],_p[2]])
    _singles = []
    _sd_singles = []
    for _item in _sample:
        if _item == 0:
            _IQ = [normal(loc=gI_mean,scale=gI_sd),normal(loc=gQ_mean,scale=gQ_sd)]
            _sdIQ = [gI_sd,gQ_sd]
        elif _item == 1:
            _IQ = [normal(loc=eI_mean,scale=eI_sd),normal(loc=eQ_mean,scale=eQ_sd)]
            _sdIQ = [eI_sd,eQ_sd]
        elif _item == 2:
            _IQ = [normal(loc=fI_mean,scale=fI_sd),normal(loc=fQ_mean,scale=fQ_sd)]
            _sdIQ = [fI_sd,fQ_sd]
        _singles.append(_IQ)
        _sd_singles.append(_sdIQ)
    if _output_type == 'single':
        return _singles,_sd_singles
    elif _output_type == 'mean':
        _mean = [np.mean([_singles[i][0] for i in range(_shots)]),np.mean([_singles[i][1] for i in range(_shots)])]
        _sd_mean = [np.sqrt(np.sum([_sd_singles[j][0]**2 for j in range(len(_sd_singles))]))/_shots,np.sqrt(np.sum([_sd_singles[j][1]**2 for j in range(len(_sd_singles))]))/_shots]
        return _mean,_sd_mean
    else :
        raise Exception('Output type not valid.')

In [ ]:
shots = 2**10

Res0,sdRes0 = readout_sim([1,0,0],shots,'single')
I0 = np.array([Res0[i][0] for i in range(shots)])
Q0 = np.array([Res0[i][1] for i in range(shots)])
Res1,sdRes1 = readout_sim([0,1,0],shots,'single')
I1 = np.array([Res1[i][0] for i in range(shots)])
Q1 = np.array([Res1[i][1] for i in range(shots)])
Res2,sdRes2 = readout_sim([0,0,1],shots,'single')
I2 = np.array([Res2[i][0] for i in range(shots)])
Q2 = np.array([Res2[i][1] for i in range(shots)])

plt.grid()
plt.scatter(I0,Q0,marker='x',label='|g>')
plt.scatter(I1,Q1,marker='x',label='|e>')
plt.scatter(I2,Q2,marker='x',label='|f>')
plt.scatter(0,0,color='black',label='O')
plt.arrow(0,0,gI_mean,gQ_mean,color='yellow',label='Aro(|g>)')
plt.arrow(0,0,eI_mean,eQ_mean,color='black',label='Aro(|e>)')
plt.arrow(0,0,fI_mean,fQ_mean,color='purple',label='Aro(|f>)')

plt.xlabel('I')
plt.ylabel('Q')
plt.legend(loc='lower right',fontsize='small')
plt.show()

In [ ]:
S0 = np.sqrt(np.mean(I0)**2+np.mean(Q0)**2)
T0 = np.arctan2(np.mean(Q0),np.mean(I0))
S1 = np.sqrt(np.mean(I1)**2+np.mean(Q1)**2)
T1 = np.arctan2(np.mean(Q1),np.mean(I1))
S2 = np.sqrt(np.mean(I2)**2+np.mean(Q2)**2)
T2 = np.arctan2(np.mean(Q2),np.mean(I2))
print(f'Aro(|g>)={S0:.4f}')
print(f'Thetaro(|g>)={T0:.4f}')
print(f'Aro(|e>)={S1:.4f}')
print(f'Thetaro(|e>)={T1:.4f}')
print(f'Aro(|f>)={S2:.4f}')
print(f'Thetaro(|f>)={T2:.4f}')

In [ ]:
# --- Teff - Pe relations ---
h = 6.62607015e-34 #[m2 kg s-1]
kB = 1.380649e-23 #[m2 kg s-2 K-1]
def Pe2Teff(_pe):
    global f01,h,kB
    return -(h*f01)/(kB*np.log(_pe/(1-_pe)))
def Teff2Pe(_Teff):
    global f01,h,kB
    return np.exp(-(h*f01)/(kB*_Teff))/(1+np.exp(-(h*f01)/(kB*_Teff)))

In [ ]:
# Experiment setup
def run_exp(_pe0,_shots_per_t,_tt):
    _state0 = np.array([1-_pe0,_pe0,0])
    _results = []
    _sd_results = []
    for k in range(len(_tt)):
        _tState = rabi_sim(_state0,_tt[k])
        _res,_sd_res = readout_sim(_tState,_shots_per_t,'mean')
        _results.append(_res)
        _sd_results.append(_sd_res)
    _I = np.array([_results[m][0] for m in range(len(_results))])
    _sdI = np.array([_sd_results[m][0] for m in range(len(_sd_results))])
    _Q = np.array([_results[m][1] for m in range(len(_results))])
    _sdQ = np.array([_sd_results[m][1] for m in range(len(_sd_results))])
    return _tt,_I,_Q,_sdI,_sdQ
    
def fit_function(x_values, y_values,y_sd, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params, y_sd)
    y_fit = function(x_values, *fitparams)
    return fitparams, y_fit, conv
def fit_function_nosd(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    return fitparams, y_fit, conv

def scale_values(_values):
    return _values-np.mean(_values)

In [ ]:
Teff = 15e-3
pe = Teff2Pe(Teff)
shots = 2**17
tt = np.arange(0,1e-6,1e-8)
t,I,Q,sdI,sdQ = run_exp(pe,shots,tt)

In [ ]:
plt.scatter(I,Q,marker='x',color='black')
plt.scatter(gI_mean,gQ_mean,color='blue',label='|g>')
# plt.scatter(eI_mean,eQ_mean,color='orange',label='|e>')
# plt.scatter(fI_mean,fQ_mean,color='green',label='|f>')
# plt.scatter(0,0,color='black',label='center')
plt.xlabel('I [~V]')
plt.ylabel('Q [~V]')
plt.legend()
plt.grid()
plt.show()

In [ ]:
N=2**(17)
print(np.mean(I))
print(np.var(I)**(1/2))
print(np.mean(Q))
print(np.var(Q)**(1/2))
print(gI_sd/np.sqrt(N))
print(gQ_sd/np.sqrt(N))

In [ ]:
signal = np.sqrt(I**2+Q**2)
sd_signal = np.sqrt((I**2*sdI**2+Q**2*sdQ**2)/(I**2+Q**2))
plt.scatter(t,signal,marker='x',color='black')
plt.xlabel('Rabi pulse duration [s]')
plt.ylabel('Readout pulse amplitude [~V]')
plt.show()

In [ ]:
fit_params, y_fit, conv = fit_function(t,
                                 signal,sd_signal, 
                                 lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),
                                 [4, 10, 250e-9, 0])
print(np.abs(fit_params[0]))
plt.errorbar(t,signal,yerr=sd_signal,fmt='.',capsize=4,ecolor='black',color='black')
plt.plot(t, y_fit)
plt.xlabel('Rabi pulse duration [s]')
plt.ylabel('Readout pulse amplitude [~V]')
plt.show()

# Hypothesis validation

### H1: $P_f\approx 0$

In [ ]:
v01 = 5e9
h = 6.62607015e-34 #[m2 kg s-1]
kB = 1.380649e-23 #[m2 kg s-2 K-1]
Temps = [15e-3,30e-3,50e-3,100e-3,300e-3]
for Temp in Temps:
    print(f'T: {Temp*1e3:.0f}mK')
    for i in range(4):
        print(f'-hv0{i:.0f}/kBT={-h*((i+1)*v01)/(kB*Temp):.1f}')

### H2: $A_\text{ref}=A_0P_g,\ A_\text{sig}=A_0P_e$

In [ ]:
peMin = Teff2Pe(15e-3)
peMax = Teff2Pe(100e-3)
Nres = 1e2
peRes = (peMax-peMin)/Nres
pes = np.arange(peMin,peMax,peRes)
amps = []
sd_amps = []
max_count = len(pes)
f = IntProgress(min=0, max=max_count)
display(f)


for j in range(len(pes)):
    shots = 2**12
    tt = np.arange(0,1e-6,1e-8)
    t,I,Q,sdI,sdQ = run_exp(pes[j],shots,tt)
    signal = np.sqrt(I**2+Q**2)
    sd_signal = np.sqrt((I**2*sdI**2+Q**2*sdQ**2)/(I**2+Q**2))
    fit_params, y_fit, conv = fit_function(t,signal,sd_signal, lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),[4, 10, 250e-9, 0])
    amps.append(np.abs(fit_params[0]))
    sd_amps.append(np.abs(conv[0][0]))
    f.value += 1


In [ ]:
fit_params, y_fit, conv = fit_function(pes,amps, sd_amps,lambda x, A, B: (A*x + B),[1, 0])
plt.scatter(pes,amps,marker='x',color='black')
# plt.errorbar(pes,amps,yerr=sd_amps,fmt='.',capsize=4,ecolor='black')
plt.plot(pes, y_fit,color='red')
plt.xlabel('Pe')
plt.ylabel('Readout pulse amplitude [~V]')
plt.show()

In [ ]:
print(f'A0: {np.abs(fit_params[0]):0.3f}')
print(f'sd(A0): {np.abs(conv[0][0])**(1/2):0.3f}')
residuals = amps - (fit_params[0]*pes + fit_params[1])
ss_res = np.sum(residuals**2)
ss_tot = np.sum((amps-np.mean(amps))**2)
r_squared = 1 - (ss_res / ss_tot)
print(f'R-squared: {r_squared:0.4f}')

In [ ]:
peMin = Teff2Pe(10e-3)
peMax = Teff2Pe(30e-3)
Nres = 1e2
peRes = (peMax-peMin)/Nres
pes = np.arange(peMin,peMax,peRes)
amps = []
sd_amps = []
max_count = len(pes)
f = IntProgress(min=0, max=max_count)
display(f)


for j in range(len(pes)):
    shots = 2**17
    tt = np.arange(0,1e-6,1e-8)
    t,I,Q,sdI,sdQ = run_exp(pes[j],shots,tt)
    signal = np.sqrt(I**2+Q**2)
    sd_signal = np.sqrt((I**2*sdI**2+Q**2*sdQ**2)/(I**2+Q**2))
    fit_params, y_fit, conv = fit_function(t,signal,sd_signal, lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),[4, 10, 250e-9, 0])
    amps.append(np.abs(fit_params[0]))
    sd_amps.append(np.abs(conv[0][0]))
    f.value += 1

In [ ]:
fit_params, y_fit, conv = fit_function(pes,amps, sd_amps,lambda x, A, B: (A*x + B),[1, 0])
plt.scatter(pes,amps,marker='x',color='black')
plt.plot(pes, y_fit,color='red')
plt.xlabel('Pe')
plt.ylabel('Readout pulse amplitude [~V]')
plt.savefig('Amps_vs_Pe_bad')
# plt.show()

In [ ]:
print(f'A0: {np.abs(fit_params[0]):0.1f}')
print(f'sd(A0): {np.abs(conv[0][0])**(1/2):0.1f}')
residuals = amps - (fit_params[0]*pes + fit_params[1])
ss_res = np.sum(residuals**2)
ss_tot = np.sum((amps-np.mean(amps))**2)
r_squared = 1 - (ss_res / ss_tot)
print(f'R-squared: {r_squared:0.4f}')

# Method validation

### Fitting curve method testing

In [ ]:
Teff = 200e-3
pe = Teff2Pe(Teff)
shots = 2**10
tt = np.arange(0,1e-6,1e-8)


In [ ]:
tRef,IRef,QRef,sdIRef,sdQRef = run_exp(1-pe,shots,tt)
plt.scatter(IRef,QRef,marker='x',color='black')
plt.scatter(gI_mean,gQ_mean,color='blue',label='|g>')
plt.scatter(eI_mean,eQ_mean,color='orange',label='|e>')
plt.scatter(fI_mean,fQ_mean,color='green',label='|f>')
plt.scatter(0,0,color='black',label='center')
plt.legend()
plt.show()

signalRef = np.sqrt(IRef**2+QRef**2)
sd_signalRef = np.sqrt((IRef**2*sdIRef**2+QRef**2*sdQRef**2)/(IRef**2+QRef**2))

fit_paramsRef, y_fitRef, convRef = fit_function(tRef,signalRef,sd_signalRef,lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),[max(signalRef)-min(signalRef), (max(signalRef)+min(signalRef))/2, 250e-9, 0])
ARef = np.abs(fit_paramsRef[0])
print(ARef)
plt.errorbar(tRef,signalRef,yerr=sd_signalRef,fmt='.',capsize=4,ecolor='blue',color='blue')
plt.plot(tRef, y_fitRef,color='blue')
plt.xlabel('Rabi pulse duration [s]')
plt.ylabel('Readout pulse amplitude [~V]')
plt.show()

In [ ]:
tSig,ISig,QSig,sdISig,sdQSig = run_exp(pe,shots,tt)
plt.scatter(ISig,QSig,marker='x',color='black')
plt.scatter(gI_mean,gQ_mean,color='blue',label='|g>')
plt.scatter(eI_mean,eQ_mean,color='orange',label='|e>')
plt.scatter(fI_mean,fQ_mean,color='green',label='|f>')
plt.scatter(0,0,color='black',label='center')
plt.legend()
plt.show()

signalSig = np.sqrt(ISig**2+QSig**2)
sd_signalSig = np.sqrt((ISig**2*sdISig**2+QSig**2*sdQSig**2)/(ISig**2+QSig**2))

fit_paramsSig, y_fitSig, convSig = fit_function(tSig,signalSig,sd_signalSig,lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),[max(signalSig)-min(signalSig), (max(signalSig)+min(signalSig))/2, 250e-9, 0])
ASig = np.abs(fit_paramsSig[0])
print(ASig)
plt.errorbar(tSig,signalSig,yerr=sd_signalSig,fmt='.',capsize=4,ecolor='red',color='red')
plt.plot(tSig, y_fitSig,color='red')
plt.xlabel('Rabi pulse duration [s]')
plt.ylabel('Readout pulse amplitude [~V]')
plt.show()

In [ ]:
Pe_exp = ASig/(ASig+ARef)
Teff_exp = Pe2Teff(Pe_exp)
print(Teff_exp)
plt.errorbar(tRef,signalRef,yerr=sd_signalRef,fmt='.',capsize=4,ecolor='blue',color='blue',label='Ref')
plt.plot(tRef, y_fitRef,color='blue')
plt.errorbar(tSig,signalSig,yerr=sd_signalSig,fmt='.',capsize=4,ecolor='red',color='red',label='Sig')
plt.plot(tSig, y_fitSig,color='red')
plt.xlabel('Rabi pulse duration [s]')
plt.ylabel('Readout pulse amplitude [~V]')
plt.legend(loc='upper right')
plt.show()

### Fitting curve method validation

In [ ]:
def run_method_fitting(_realTeff,_shots_per_t,_tt):
    global h,kB,f01
    realPe0 = Teff2Pe(_realTeff)
    tRef,IRef,QRef,sdIRef,sdQRef = run_exp(1-realPe0,_shots_per_t,_tt)
    signalRef = np.sqrt(IRef**2+QRef**2)
    sd_signalRef = np.sqrt((IRef**2*sdIRef**2+QRef**2*sdQRef**2)/(IRef**2+QRef**2))
    fit_paramsRef, y_fitRef, convRef = fit_function(tRef,signalRef,sd_signalRef, lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),[max(signalRef)-min(signalRef), (max(signalRef)+min(signalRef))/2, 250e-9, 0])
    ARef = np.abs(fit_paramsRef[0])
    sdARef = np.sqrt(np.abs(convRef[0][0]))

    tSig,ISig,QSig,sdISig,sdQSig = run_exp(realPe0,_shots_per_t,_tt)
    signalSig = np.sqrt(ISig**2+QSig**2)
    sd_signalSig = np.sqrt((ISig**2*sdISig**2+QSig**2*sdQSig**2)/(ISig**2+QSig**2))
    fit_paramsSig, y_fitSig, convSig = fit_function(tSig,signalSig,sd_signalSig, lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),[max(signalSig)-min(signalSig), (max(signalSig)+min(signalSig))/2, 250e-9, 0])
    ASig = np.abs(fit_paramsSig[0])
    sdASig = np.sqrt(np.abs(convSig[0][0]))

    _Pe_exp = ASig/(ASig+ARef)
    _sd_Pe_exp = np.sqrt((ARef**2*sdASig**2)/(ASig**2+ARef**2)**4 + (ASig**2*sdARef**2)/(ASig**2+ARef**2)**4)
    _Teff_exp = Pe2Teff(_Pe_exp)
    _sd_Teff_exp = np.abs((h*f01)/(kB*(np.log( _Pe_exp/(1- _Pe_exp)))**2*_Pe_exp*(1-_Pe_exp)))*_sd_Pe_exp
    return _Teff_exp, _sd_Teff_exp, _Pe_exp, _sd_Pe_exp

In [ ]:
real_Teff = 15e-3

N_meas = 10

shots = 5000
tt = np.arange(0,0.6e-6,1e-7)
f = IntProgress(min=0, max=N_meas)
display(f)

meas = []
for n in range(N_meas):
    meas.append(run_method_fitting(real_Teff,shots,tt))
    f.value += 1

In [ ]:
exp_Teff = np.mean(np.array([meas[i][0] for i in range(len(meas))]))
sd_exp_Teff = np.sqrt(np.sum((np.array([meas[i][0] for i in range(len(meas))])-exp_Teff)**2)/len(meas))

In [ ]:
print(f'Real Teff: {real_Teff*1e3:.1f} mK')
print(f'Exp Teff: {exp_Teff*1e3:.1f} mK')
print(f'sd Exp Teff: {sd_exp_Teff*1e3:.1f} mK')

In [ ]:
real_Teffs = np.arange(15e-3,100e-3,10e-3)
shots = 2**17
tt = np.arange(0,1e-6,1e-8)
exp_Teffs = []
sd_exp_Teffs = []
exp_Pes = []
sd_exp_Pes = []


max_count = len(real_Teffs)
f = IntProgress(min=0, max=max_count)
display(f)

for l in range(len(real_Teffs)):
    exp_Teff,sd_exp_Teff,exp_Pe,sd_exp_Pe = run_method_fitting(real_Teffs[l],shots,tt)
    exp_Teffs.append(exp_Teff)
    sd_exp_Teffs.append(sd_exp_Teff)
    exp_Pes.append(exp_Pe)
    sd_exp_Pes.append(sd_exp_Pe)
    f.value += 1

In [ ]:
tteff = np.arange(15e-3,100e-3,1e-3)
ppe = Teff2Pe(tteff)

plt.errorbar(np.array(real_Teffs)*1e3,np.array(exp_Pes)*1e2,yerr=np.array(sd_exp_Pe)*1e2,fmt='.',capsize=4,ecolor='black',label='Exp Pe')
plt.plot(tteff*1e3,ppe*1e2,'-',color='red',label='MB dist')
plt.plot([10,95],[0.1,0.1],'--',color='purple')
plt.xlabel('Real Teff [mK]')
plt.ylabel('Measured Pe [%]')
plt.show()

### Min-Max method testing

In [ ]:
real_Teff = 15e-3
shots = 4000
C = 2000
realPe0 = Teff2Pe(real_Teff)
_tt = np.array([TRabi/2,TRabi])

measRef = []
sd_measRef = []
f = IntProgress(min=0, max=C)
display(f)
for c in range(C):
    tRef,IRef,QRef,sdIRef,sdQRef = run_exp(1-realPe0,shots,_tt)
    signalRef = np.sqrt(IRef**2+QRef**2)
    sd_signalRef = np.sqrt((IRef**2*sdIRef**2+QRef**2*sdQRef**2)/(IRef**2+QRef**2))
    measRef.append(signalRef[0] - signalRef[1])
    sd_measRef.append(np.sqrt(sd_signalRef[0]**2 + sd_signalRef[1]**2))
    f.value += 1

ARef = np.mean(np.array(measRef))
sdARef_typeA = np.sqrt(np.sum(np.array(sd_measRef)**2)/C**2)
sdARef_typeB = np.sqrt(np.sum((np.array(measRef)-ARef)**2)/(C))
sdARef = np.sqrt(sdARef_typeA**2 + sdARef_typeB**2)
print(f'ARef: {ARef:.3f}')
print(f'sdARef: {sdARef:.3f}')

In [ ]:
measSig = []
sd_measSig = []
f = IntProgress(min=0, max=C)
display(f)
for c in range(C):
    tSig,ISig,QSig,sdISig,sdQSig = run_exp(realPe0,shots,_tt)
    signalSig = np.sqrt(ISig**2+QSig**2)
    sd_signalSig = np.sqrt((ISig**2*sdISig**2+QSig**2*sdQSig**2)/(ISig**2+QSig**2))
    measSig.append(signalSig[0] - signalSig[1])
    sd_measSig.append(np.sqrt(sd_signalSig[0]**2 + sd_signalSig[1]**2))
    f.value += 1

ASig = np.mean(np.array(measSig))
sdASig_typeA = np.sqrt(np.sum(np.array(sd_measSig)**2)/C**2)
sdASig_typeB = np.sqrt(np.sum((np.array(measSig)-ASig)**2)/(C))
sdASig = np.sqrt(sdASig_typeA**2 + sdASig_typeB**2)
print(f'ASig: {ASig:.6f}')
print(f'sdASig: {sdASig:.3f}')

In [ ]:
Pe_exp = ASig/(ASig+ARef)
print(Pe_exp)
sd_Pe_exp = np.sqrt((ARef**2*sdASig**2)/(ASig**2+ARef**2)**4 + (ASig**2*sdARef**2)/(ASig**2+ARef**2)**4)
print(sd_Pe_exp)
Teff_exp = Pe2Teff(Pe_exp)
print(Teff_exp)
sd_Teff_exp = np.abs((h*f01)/(kB*(np.log( Pe_exp/(1- Pe_exp)))**2*Pe_exp*(1-Pe_exp)))*sd_Pe_exp
print(sd_Teff_exp)

### Min_Max method validation

In [ ]:
def run_method_minmax(_realTeff,_shots_per_t,_cycles):
    global h,kB,f01,TRabi
    realPe0 = Teff2Pe(_realTeff)
    _tt = np.array([TRabi/2,TRabi])
    measRef = []
    sd_measRef = []
    f = IntProgress(min=0, max=_cycles)
    display(f)
    for c in range(_cycles):
        tRef,IRef,QRef,sdIRef,sdQRef = run_exp(1-realPe0,_shots_per_t,_tt)
        signalRef = np.sqrt(IRef**2+QRef**2)
        sd_signalRef = np.sqrt((IRef**2*sdIRef**2+QRef**2*sdQRef**2)/(IRef**2+QRef**2))
        measRef.append(signalRef[0] - signalRef[1])
        sd_measRef.append(np.sqrt(sd_signalRef[0]**2 + sd_signalRef[1]**2))
        f.value += 1
    
    ARef = np.mean(np.array(measRef))
    sdARef_typeA = np.sqrt(np.sum(np.array(sd_measRef)**2)/_cycles**2)
    sdARef_typeB = np.sqrt(np.sum((np.array(measRef)-ARef)**2)/(_cycles))
    sdARef = np.sqrt(sdARef_typeA**2 + sdARef_typeB**2)
                        
    measSig = []
    sd_measSig = []
    f = IntProgress(min=0, max=_cycles)
    display(f)
    for c in range(C):
        tSig,ISig,QSig,sdISig,sdQSig = run_exp(realPe0,_shots_per_t,_tt)
        signalSig = np.sqrt(ISig**2+QSig**2)
        sd_signalSig = np.sqrt((ISig**2*sdISig**2+QSig**2*sdQSig**2)/(ISig**2+QSig**2))
        measSig.append(signalSig[0] - signalSig[1])
        sd_measSig.append(np.sqrt(sd_signalSig[0]**2 + sd_signalSig[1]**2))
        f.value += 1
    ASig = np.mean(np.array(measSig))
    sdASig_typeA = np.sqrt(np.sum(np.array(sd_measSig)**2)/_cycles**2)
    sdASig_typeB = np.sqrt(np.sum((np.array(measSig)-ASig)**2)/(_cycles))
    sdASig = np.sqrt(sdASig_typeA**2 + sdASig_typeB**2)

    _Pe_exp = ASig/(ASig+ARef)
    _sd_Pe_exp = np.sqrt((ARef**2*sdASig**2)/(ASig**2+ARef**2)**4 + (ASig**2*sdARef**2)/(ASig**2+ARef**2)**4)
    _Teff_exp = Pe2Teff(_Pe_exp)
    _sd_Teff_exp = np.abs((h*f01)/(kB*(np.log( _Pe_exp/(1- _Pe_exp)))**2*_Pe_exp*(1-_Pe_exp)))*_sd_Pe_exp
    return [_Teff_exp, _sd_Teff_exp, _Pe_exp, _sd_Pe_exp]

In [ ]:
real_Teff = 15e-3
shots = 5000
C = 10000
Teff_exp, sd_Teff_exp, Pe_exp, sd_Pe_exp=run_method_minmax(real_Teff,shots,C)

In [ ]:
print(f'Pe_exp: {Pe_exp:.7f}')
print(f'sdPe_exp: {sd_Pe_exp:.7f}')
print(f'Teff_exp: {Teff_exp:.3f}')
print(f'sdTeff_exp: {sd_Teff_exp:.3f}')

In [ ]:
Teffs = np.arange(15e-3,40e-3,5e-3)
real_Teff = 15e-3
shots = 5000
C = 10000
measurements = []
for u in range(len(Teffs)):
    print(f'{Teffs[u]*1e3:.0f} mK')
    measurements.append(run_method_minmax(Teffs[u],shots,C))

In [ ]:
Teffs_exp = [measurements[i][0] for i in range(len(measurements))]
sdTeffs_exp = [measurements[i][1] for i in range(len(measurements))]
Pes_exp = [measurements[i][2] for i in range(len(measurements))]
sdPes_exp = [measurements[i][3] for i in range(len(measurements))]

In [ ]:
tteff = np.arange(10e-3,40e-3,1e-3)
ppe = Teff2Pe(tteff)

plt.errorbar(np.array(Teffs)*1e3,np.array(Pes_exp)*1e2,yerr=np.array(sdPes_exp )*1e2,fmt='.',capsize=4,ecolor='black',label='Exp Pe')
plt.plot(tteff*1e3,ppe*1e2,'-',color='red',label='Pe MB dist')
plt.plot([10,40],[0.1,0.1],'--',color='purple',label='Pe=0.1%')
plt.xlabel('Real Teff [mK]')
plt.ylabel('Measured Pe [%]')
plt.legend()
plt.show()